In [4]:
!pip install SpeechRecognition
!pip install pyaudio

  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/82/d8/f043c854aad450a76e476b0cf9cda1956419e1dacf1062eb9df3c0055abe/PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
   --------- ----------------------------- 41.0/164.1 kB 393.8 kB/s eta 0:00:01
   ---------------------------------------- 164.1/164.1 kB 1.2 MB/s eta 0:00:00


In [1]:
pip install textblob


  Obtaining dependency information for textblob from https://files.pythonhosted.org/packages/02/07/5fd2945356dd839974d3a25de8a142dc37293c21315729a41e775b5f3569/textblob-0.18.0.post0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------- ----------- 440.3/626.3 kB 9.2 MB/s eta 0:00:01
   ---------------------------------------  624.6/626.3 kB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 626.3/626.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
import cv2
from deepface import DeepFace
import speech_recognition as sr
from textblob import TextBlob
# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize speech recognizer
recognizer = sr.Recognizer()

# Initialize speech timeout counter
speech_timeout = 0

while True:
    ret, frame = cap.read()
    if not ret:  
        continue  # Skip to the next iteration if the frame is invalid

    # Facial expression recognition
    try:
        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        print("Result:", result)  # Print the result to understand its structure
        facial_emotion = result[0]['dominant_emotion']
    except Exception as e:
        print("Facial expression recognition error:", str(e))
        facial_emotion = "Unknown"

    # Speech emotion recognition
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio_data = recognizer.listen(source, timeout=5)  # Set timeout for speech recognition
            speech_text = recognizer.recognize_google(audio_data)
            print("Speech detected:", speech_text)
            # Perform speech emotion recognition here
            
            blob = TextBlob(speech_text)
            sentiment_score = blob.sentiment.polarity

            # Classify sentiment
            if sentiment_score > 0:
                 speech_emotion = "Positive"
            elif sentiment_score < 0:
                 speech_emotion = "Negative"
            else:
                 speech_emotion = "Neutral"
            
            speech_timeout = 0  # Reset speech timeout counter
        except sr.WaitTimeoutError:
            print("No speech detected.")
            speech_emotion = "Unknown"
            speech_timeout += 1
            if speech_timeout >= 3:  # If no speech detected for 3 consecutive iterations, break the loop
                print("Speech timeout reached. Exiting.")
                break
        except Exception as e:
            print("Speech recognition error:", str(e))
            speech_emotion = "Unknown"

    # Combine and display results
    combined_emotion = f"Facial: {facial_emotion} | Speech: {speech_emotion}"
    cv2.putText(frame, combined_emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the resulting frame
    cv2.imshow('Multimodal Emotion Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


Result: [{'emotion': {'angry': 0.03338118748403713, 'disgust': 1.6623164947270755e-08, 'fear': 0.09717263162047456, 'happy': 0.001976269895309837, 'sad': 0.11854407618457533, 'surprise': 0.0014945933780378905, 'neutral': 99.74743126317449}, 'dominant_emotion': 'neutral', 'region': {'x': 297, 'y': 78, 'w': 179, 'h': 179}, 'face_confidence': 9.194292111962568}]
Listening...
Speech detected: am I visible in audible to you
Result: [{'emotion': {'angry': 0.44353859799640205, 'disgust': 1.2057751136783457e-07, 'fear': 0.04056102578359535, 'happy': 0.020226033438060172, 'sad': 0.2689013169898013, 'surprise': 0.007534104054220655, 'neutral': 99.21923289069737}, 'dominant_emotion': 'neutral', 'region': {'x': 287, 'y': 83, 'w': 263, 'h': 263}, 'face_confidence': 9.209602049493697}]
Listening...
Speech detected: I want to tell you something right now
Result: [{'emotion': {'angry': 0.0951844445116735, 'disgust': 3.6163661500894687e-09, 'fear': 0.03195902591966528, 'happy': 0.05178904940467484, 's